# Load embeddings 

In [3]:
from transformers import AutoModel
import sys
sys.path.append('/scratch/dengm/distent/Disentangle-features/archive')
from features import *
from metrics import *
import os
import torch as t
import torch.nn as nn
import pickle
from transformers import AutoTokenizer
import numpy as np
import torch as t
def get_acti(samples, emb, tokenizer):
    idList = np.random.choice(emb.shape[0], samples, replace = False)
    words = tokenizer.convert_ids_to_tokens(idList)
    embs = emb[idList]
    return embs, words
model = AutoModel.from_pretrained("prajjwal1/bert-tiny")
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
emb = model.embeddings.word_embeddings.weight.detach().numpy()
embs, words = get_acti(emb.shape[0], emb, tokenizer)
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
device = t.device("cuda" if t.cuda.is_available() else "cpu")

np.save(f"bert_tiny_embs.npy", embs)
np.save(f"bert_tiny_words.npy", words)
embs = np.load(f"bert_tiny_embs.npy")
words = np.load(f"bert_tiny_words.npy")

print(embs.shape, words.shape)



Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(30522, 128) (30522,)


# Solve for features

In [4]:
f1, e1, inf1 = wrapper('gd', 'bert_task', t.Tensor(embs).to(device), init_lamb = 0.15, lamb_left = 0.4, lamb_right = 0.6, guess_factor=32, lr = 5e-3, steps = 8000)

torch.Size([30522, 128])


100%|██████████| 8000/8000 [05:00<00:00, 26.64it/s, accuracy=0, loss=6.02e+3]


NameError: name 'norm_ord' is not defined

In [9]:
print(inf1, metric_total_acti(f1))

def norminfty(x):
    return x.abs().max(dim = 1)[0] + 1e-8
def normp(p):
    return lambda x: x.abs().pow(p).sum(dim = 1)
def norm_ord(x, p):
    return normp(p)(x)
def sparse_metric_scaled(pw):
    return lambda feat: (norm_ord(feat, pw) / norminfty(feat) ** pw).mean() * (1 + pw) - pw

def sparse_metric_mean_scaled(pw):
    return lambda feat: (norm_ord(feat, pw).mean() / norminfty(feat).mean() ** pw) * (1 + pw) - pw

def sparse_metric_mean(pw):
    return lambda feat: (norm_ord(feat, pw).mean() / norminfty(feat).mean() ** pw)
print(sparse_metric_mean_scaled(1)(f1))
print(sparse_metric_mean_scaled(0.7)(f1))
print(sparse_metric_mean(1)(f1))
print(sparse_metric_mean(0.7)(f1))
# print(inf1, metric_total_acti(f1))

{'loss': 6002.88671875, 'loss_est': 5.228413678658825, 'lamb': 0.15, 'maxA': 0.250776, 'guess_factor': 32, 'negative_penalty': 1, 'lr': 0.005, 'steps': 8000, 'size': 30522} 3.1832583
tensor(5.3665)
tensor(13.5756)
tensor(3.1833)
tensor(8.3974)


In [16]:

sum_feat = f1.sum(dim = 0)
best_dims = sum_feat.argsort(descending = True)
for i in range(20, 4000, 100):
    print(i, sum_feat[best_dims[i]].item())
    print(get_feat_top(words, f1, best_dims[i], (f1[:, best_dims[i]] > 0.05).sum())[0])
    print()

20 19.706186294555664
['maintains', 'employs', 'conducts', 'provides', 'produces', 'publishes', 'enjoys', 'organizes', 'boasts', 'offers', 'promotes', 'operates', 'generates', 'contributes', 'manages', 'delivers', 'owns', 'performs', 'oversees', 'elects', 'develops', 'utilizes', 'collects', 'sells', 'possesses', 'supports', 'serves', 'holds', 'participates', 'creates', 'has', 'manufactures', 'builds', 'selects', 'incorporates', 'teaches', 'prefers', 'receives', 'lacks', 'includes', 'encompasses', 'comprises', 'regulates', 'contains', 'occupies', 'attracts', 'gathers', 'specializes', 'retains', 'focuses', 'defines', 'lays', 'wears', 'represents', 'advises', 'uses', 'prepares', 'establishes', 'assigns', 'relies', 'attends', 'investigates', 'weighs', 'seeks', 'sits', 'combines', 'competes', 'consists', 'presents', 'writes', 'accepts', 'gives', 'eats', 'celebrates', 'carries', 'ensures', 'facilitates', 'requires', 'airs', 'specifies', 'resides', '##ises', 'draws', 'adds', 'applies', 'handl

In [21]:

sum_feat = t.tensor(embs).sum(dim = 0)
best_dims = sum_feat.argsort(descending = True)
for i in range(0, 128, 1):
    print(i, sum_feat[best_dims[i]].item())
    print(get_feat_top(words, t.tensor(embs), best_dims[i], 20)[0])
    print()

0 2260.555908203125
['[unused751]', '[unused264]', 'ধ', '##秋', '[unused699]', '##谷', '[unused351]', '[unused967]', '[unused140]', '[unused76]', '[unused955]', '[unused801]', '[unused152]', '[unused497]', 'ٹ', '##都', '[unused425]', '[unused171]', '[unused12]', '##定']

1 1605.2452392578125
['ट', 'ћ', 'љ', '##ˤ', '##氷', '[unused788]', '##侍', 'थ', '##口', '¶', '[unused237]', 'ள', '♣', '¤', 'ɹ', '[unused840]', '⇒', '##movable', 'ь', '##正']

2 1478.56640625
['covent', 'wrestlemania', 'precedence', 'disagreement', 'ridden', 'eternity', 'fourier', '1016', 'mistaken', 'stevie', 'disabilities', 'teaming', 'sometime', 'intention', 'disagreements', 'notoriety', 'bipolar', 'volvo', 'partnering', 'debating']

3 1410.4510498046875
['##₆', '##viere', '##urized', '##евич', 'ringo', 'woodrow', '##eville', 'claude', '##₅', '##naud', 'ℓ', 'letterman', 'topological', '##♯', '##iance', 'francois', 'baptiste', 'ⁱ', 'brigham', '##minster']

4 1353.4869384765625
['##⋅', 'ˈ', '##pts', '##ٹ', '₍', '##guide', '##ˈ

In [23]:

from sklearn.decomposition import PCA
pca = PCA(n_components=embs.shape[1])
pca.fit(embs)
new_coords = pca.transform(embs)
sum_feat = t.tensor(new_coords).sum(dim = 0)
best_dims = sum_feat.argsort(descending = True)
for i in range(0, 128, 1):
    print(i, sum_feat[best_dims[i]].item())
    print(get_feat_top(words, t.Tensor(new_coords).to(device), best_dims[i], 20)[0])
    print()

0 0.005957603454589844
['##－', 'ᅦ', 'տ', '[unused164]', '[unused494]', '[unused326]', '[unused751]', '##主', '[unused899]', '[unused209]', '[unused3]', 'ₗ', '[unused933]', '[unused14]', '[unused179]', '[unused749]', '彳', 'კ', '[unused955]', '[unused559]']

1 0.005356550216674805
['throughout', 'to', 'in', '##nostic', 'anymore', 'toward', 'across', '##dened', 'beside', 'aback', 'towards', 'during', 'behind', 'around', 'beforehand', 'for', 'into', 'against', 'between', '##nted']

2 0.005305767059326172
['a', 'is', 'was', 'an', '"', 'the', 'are', 'its', 'were', 'being', 'another', 'it', 'becomes', 'be', 'those', 'became', 'been', 'this', 'your', 'pont']

3 0.004507541656494141
['they', 'there', 'it', 'though', 'she', 'others', 'besides', 'nonetheless', 'consequently', 'therefore', 'however', 'he', 'powerplant', 'importantly', 'alternatively', 'although', 'meantime', 'snails', 'apart', 'indeed']

4 0.004450321197509766
['began', 'ceased', '##uen', '##week', '##nal', 'begun', '##rift', 'ende

# Visualize the directions of a given token

In [62]:

def visualize_by_features(feat, words, word_id):
    print(words[word_id])
    acts = feat[word_id].argsort(descending = True)
    for j in range(3):
        # display all the activations contribute > 1/20 to the sum
        total_act = feat[word_id].sum()
        print(acts[j].item(), feat[word_id, acts[j]].item(), feat[:, acts[j]].sum().item())
        print(get_feat_top(words, feat, acts[j], 20)[0])

def visualize_nearest(feat, words, word_id):
    # get the nearest 10 words
    diff = feat - feat[word_id]
    dist = diff.pow(2).sum(dim = 1)
    nearest = dist.argsort()
    for i in range(20):
        print(words[nearest[i]], dist[nearest[i]].item())

In [45]:
visualize_by_features(f1, words, list(words).index('peking'))

peking
2147 0.18776926398277283 11.760446548461914
['zhejiang', 'guangdong', 'jiangsu', 'hunan', 'shandong', 'henan', 'fujian', 'sichuan', 'tianjin', 'guangzhou', 'yunnan', 'xinjiang', 'shenzhen', 'nanjing', 'taipei', 'beijing', 'shanghai', 'china', 'peking', 'taiwan']
507 0.13848425447940826 11.693121910095215
['harvard', 'yale', 'rutgers', 'stanford', 'princeton', 'tulane', 'georgetown', 'fordham', 'purdue', 'cornell', 'mcgill', 'emory', 'cambridge', 'tufts', 'oxford', 'baylor', 'loyola', 'concordia', 'syracuse', 'clemson']
3786 0.1294572502374649 13.923301696777344
['bucharest', 'istanbul', 'tbilisi', 'seoul', 'tehran', 'helsinki', 'budapest', 'beijing', 'yerevan', 'oslo', 'zagreb', 'belgrade', 'tallinn', 'brussels', 'jakarta', 'nanjing', 'cairo', 'tokyo', 'gothenburg', 'beirut']


In [63]:
visualize_nearest(f1, words, list(words).index('peking'))

peking 0.0
shanghai 0.05828685313463211
beijing 0.05916265770792961
taiwan 0.06983471661806107
nanjing 0.07084673643112183
shenzhen 0.07876468449831009
guangzhou 0.09129001200199127
yunnan 0.10201356559991837
baylor 0.10239443928003311
tianjin 0.10664130002260208
concordia 0.10816189646720886
berlin 0.10853046923875809
clemson 0.10898559540510178
auburn 0.10986308008432388
hopkins 0.11097830533981323
using 0.11136411875486374
hero 0.11137408018112183
field 0.1115109845995903
china 0.11199554800987244
business 0.11212232708930969


In [58]:
visualize_by_features(f1, words, list(words).index('physics'))

physics
2819 0.16798430681228638 8.360572814941406
['journalism', 'philosophy', 'mathematics', 'literature', 'arts', 'theology', 'economics', 'architecture', 'poetry', 'logic', 'physics', 'medicine', 'engineering', 'art', 'science', 'law', 'divinity', 'astronomy', 'humanities', 'music']
3550 0.1636885702610016 13.366540908813477
['zoology', 'biology', 'biochemistry', 'physiology', 'genetics', 'botany', 'geology', 'sociology', 'anatomy', '##biology', 'pathology', 'anthropology', 'linguistics', 'chemistry', 'ecology', '##physics', 'geography', 'philology', 'dentistry', 'psychology']
3303 0.12513267993927002 4.387062072753906
['experiments', 'experiment', 'particle', 'einstein', 'experimentation', 'vacuum', 'physics', 'relativity', 'physicist', '##utter', 'experimenting', 'simulate', 'analogy', 'theoretical', 'experimental', 'tested', 'invented', 'particles', 'tests', 'diagram']


In [ ]:
visualize_nearest(f1, words, list(words).index('physics'))